# Lab 1: Building the Governance Foundation

In this hands-on lab, you'll learn how to build AI agents that respect data governance policies using Databricks Unity Catalog. We'll create an HR Analyst agent that can answer questions about employees, compensation, and policies while automatically enforcing security controls through governance and identity best practices.

### Lab Structure
**Lab 1: Building the Governance Foundation**
- Set up
    - Make sure you are connected to `serverless` compute
    - Create HR tables in `clientcare.hr_data` from GitHub
    - Apply data classifications using Unity Catalog tags
    - Create a view with anonymization and filtering
    - Create a `Devs` Group and `hr_data_analysts` service principal
    - Grant appropriate permissions on our view 
    - Implement table-level column masking for SSN
    - Build functions for querying tables

**Lab 2: Creating the AI Agent**
- Compile the HR Analyst agent 
- Bind the LLM, system prompt, and tools 

**Lab 3: Agent Evaluation and Testing**
- Register the agent model in MLflow
- Evaluate agent responses using LLM as a judge
- Deploy the agent as a job on behalf of the service principal
- Test the complete governance stack

### **Initial Setup and Data Verification**

Let's start by verifying our environment and examining the data classifications we've already applied.

_Background for those new to Databricks_

- We use `spark.sql()` for all governance OPERATIONS (CREATE VIEW, GRANT, etc.)
- We use `pandas.DataFrame()` for showing results in a nice format
- We use the Databricks SDK (`WorkspaceClient`) for workspace verification

**Implementing Unity Catalog Governance: Available Methods**

1. **Pure SQL** - The standard approach for production environments 
2. **Spark SQL in Python** - SQL commands as strings in Python (what we'll use)
3. **PySpark DataFrame API** - The Pythonic approach for queries
4. **Databricks SDK** - For reading data, automation and verification
5. **REST API** - For external integrations and CI/CD pipelines
6. **Terraform** - Infrastructure as Code approach for version-controlled, repeatable deployments

In this lab, we'll primarily use **Spark SQL in Python** (#2) for governance operations and the **SDK** (#4) for verification.

In [0]:
# Set our working environment

from databricks.sdk import WorkspaceClient
import pandas as pd

# Use the workspace client to retrieve information about the current user
w = WorkspaceClient()

# Catalog and schema names
catalog_name = "clientcare"
schema_name = "hr_data"

# Create the catalog if it does not exist
spark.sql(
    f"CREATE CATALOG IF NOT EXISTS {catalog_name}"
)

# Create the schema in the catalog
spark.sql(
    f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}"
)

# Set widgets for catalog and schema
dbutils.widgets.text("catalog_name", defaultValue=catalog_name, label="Catalog Name")
dbutils.widgets.text("schema_name", defaultValue=schema_name, label="Schema Name")

In [0]:
import requests
import io

base_url = "https://raw.githubusercontent.com/databricks/tmm/main/Governance-lab/data"
csv_files = {
    "compensation_data": f"{base_url}/compensation_data.csv",
    "employee_records": f"{base_url}/employee_records.csv", 
    "hr_cases": f"{base_url}/hr_cases.csv",
    "internal_procedures": f"{base_url}/internal_procedures.csv",
    "performance_reviews": f"{base_url}/performance_reviews.csv",
    "public_policies": f"{base_url}/public_policies.csv"
}

# Download and load each CSV file
for table_name, url in csv_files.items():
    # Download CSV data
    response = requests.get(url)
    response.raise_for_status()
    
    # Read CSV into pandas DataFrame
    df = pd.read_csv(io.StringIO(response.text))
    
    # Convert to Spark DataFrame and write to table
    spark_df = spark.createDataFrame(df)
    spark_df.write.mode("overwrite").saveAsTable(f"{catalog_name}.hr_data.{table_name}")
print("Tables created successfully")

Tables created successfully


In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")
print(f"{catalog_name} catalog and {schema_name} schema have been created")


clientcare catalog and hr_data schema have been created


In [0]:

# This demonstrates the current security risk - without proper governance controls,
# all sensitive data (SSNs, salaries, bonuses) is fully exposed to any user with table access
# In a real production environment, this would be a major compliance violation

display(spark.sql("""
   SELECT 
       e.employee_id,
       e.first_name,
       e.last_name,
       e.department,
       e.ssn,
       e.phone,
       e.email,
       e.hire_date,
       c.base_salary,
       c.bonus,
       c.stock_options
   FROM employee_records e
   JOIN compensation_data c ON e.employee_id = c.employee_id
   ORDER BY e.employee_id
   LIMIT 10
"""))

employee_id,first_name,last_name,department,ssn,phone,email,hire_date,base_salary,bonus,stock_options
E001,John,Smith,Engineering,123-45-6789,555-0101,john.smith@company.com,2020-01-15,120000,15000,5000
E002,Sarah,Johnson,Sales,234-56-7890,555-0102,sarah.j@company.com,2019-03-22,85000,12000,2000
E003,Michael,Brown,HR,345-67-8901,555-0103,m.brown@company.com,2021-06-10,95000,10000,3000
E004,Emily,Davis,Engineering,456-78-9012,555-0104,emily.d@company.com,2022-02-28,110000,14000,4000
E005,Robert,Wilson,Marketing,567-89-0123,555-0105,r.wilson@company.com,2020-11-05,90000,11000,2500
E006,Lisa,Anderson,Sales,678-90-1234,555-0106,l.anderson@company.com,2021-08-14,88000,11500,2200
E007,David,Taylor,HR,789-01-2345,555-0107,d.taylor@company.com,2019-12-03,92000,9500,2800
E008,Jennifer,Martin,Engineering,890-12-3456,555-0108,j.martin@company.com,2023-01-20,105000,13000,3500
E009,William,Garcia,Marketing,901-23-4567,555-0109,w.garcia@company.com,2022-05-17,87000,10500,2400
E010,Maria,Rodriguez,Finance,012-34-5678,555-0110,m.rodriguez@company.com,2020-09-08,115000,16000,4500


### **Apply Data Classifications and Tags**

First, we'll tag our tables with sensitivity levels. Table properties are metadata tags that help classify data sensitivity without changing the actual data structure:

- Metadata key-value pairs attached to tables
- Used by governance tools to understand data sensitivity
- Don't affect table structure or data
- Can be queried programmatically for compliance

Our data classification schema:
 - Public: Anyone can access (company policies)
 - Internal: Employees only (procedures)
 - Confidential: Limited access (employee records, reviews)
 - Restricted: Highly sensitive (compensation, HR cases)

Adding classification tags like "Confidential," "Restricted," or "Public" to Unity Catalog serve as metadata that make your data assets more manageable, secure, and compliant with both internal policies and external regulations.


In [0]:
# Define the classification tags for each table
tables_config = [
    ("clientcare.hr_data.employee_records", "Confidential", "true"),
    ("clientcare.hr_data.compensation_data", "Restricted", "true"),
    ("clientcare.hr_data.performance_reviews", "Confidential", "true"),
    ("clientcare.hr_data.hr_cases", "Restricted", "true"),
    ("clientcare.hr_data.public_policies", "Public", "false"),
    ("clientcare.hr_data.internal_procedures", "Internal", "false")
]

# Apply classifications using tags (tags are metadata)
for table_name, classification, has_pii in tables_config:
    spark.sql(f"""
        ALTER TABLE {table_name} SET TAGS (
            'classification' = '{classification}',
            'contains_pii' = '{has_pii}'
        )
    """)
    print(f"✓ Classified {table_name} as {classification}")

✓ Classified clientcare.hr_data.employee_records as Confidential
✓ Classified clientcare.hr_data.compensation_data as Restricted
✓ Classified clientcare.hr_data.performance_reviews as Confidential
✓ Classified clientcare.hr_data.hr_cases as Restricted
✓ Classified clientcare.hr_data.public_policies as Public
✓ Classified clientcare.hr_data.internal_procedures as Internal


**Agent Permissions & Agent Access Requirements**
Now that we've classified our data, let's design the right access level for our HR analytics agent through purpose-built views.

**Key Decision:** The agent needs a specialized view - "Data Analyst View" - that provides analytical capabilities while protecting individual privacy.

What the Agent Needs vs. Doesn't Need:
| Data Type | HR Admin Sees | Manager Sees | Data Analyst | Why? |
|-----------|---------------|--------------|------------------|------|
| **Names** | ✅ John Smith | ✅ John Smith | ❌ Anonymous IDs | Prevents bias, protects privacy |
| **SSN** | ✅ 123-45-6789 | ❌ Hidden | ❌ Hidden | No analytical value |
| **Salary** | ✅ $120,000 | ❌ Hidden | ✅ $120,000 | Needed for accurate statistics |
| **Department** | ✅ Engineering | ✅ Engineering | ✅ Engineering | Needed for grouping |
| **Performance** | ✅ 4.5 | ✅ 4.5 | ✅ 4.5 | Needed for correlation analysis |
| **Email** | ✅ john@company.com | ✅ john@company.com | ❌ Hidden | No analytical value |
| **Phone** | ✅ 555-1234 | ✅ 555-1234 | ❌ Hidden | No analytical value |

The Agent's Mission:
Answer questions like:
- "What's the salary distribution in Engineering?"
- "Is there pay equity across departments?"
- "What's the correlation between performance and compensation?"

WITHOUT being able to answer:
- "What's John Smith's salary?"
- "Who are the top 5 highest paid employees?"
- "Show me SSNs for employees making over $100k"

### **Create Classification-Aware View**

Now we'll create the `data_analyst_view` that automatically provide appropriate data access based on our classifications. This view designed for analysts will have Anonymous IDs + full compensation data for statistical analysis.

**Why Views for Access Control:**
- **Simplicity**: Instead of complex masking rules for every possible user type, create purpose-built views
- **Clarity**: Each view has a clear business purpose and user type
- **Maintainability**: Easier to understand and modify than intricate permission matrices
- **Performance**: Views are optimized SQL - no runtime masking overhead
- **Security Layer**: In Databricks, views act as your security layer since we grant permissions directly to principals

In [0]:
# DATA ANALYST VIEW - For statistical analysis 

# Key features: Anonymous employee IDs, full salary access, excludes Legal dept
spark.sql("""
CREATE OR REPLACE VIEW data_analyst_view AS
SELECT 
    CONCAT('EMP_', LPAD(e.employee_id, 6, '0')) as anonymous_id,  -- EMP_000001 format
    e.department,
    YEAR(e.hire_date) as hire_year,                            -- Year only for better anonymization
    c.base_salary,                                              -- Full salary for analytics
    c.bonus,
    c.stock_options,
    YEAR(c.effective_date) as comp_year,
    pr.rating,
    QUARTER(pr.review_date) as review_quarter,
    YEAR(pr.review_date) as review_year
    -- Removed pr.comments to prevent identifying information
FROM employee_records e
LEFT JOIN compensation_data c ON e.employee_id = c.employee_id
LEFT JOIN performance_reviews pr ON e.employee_id = pr.employee_id
WHERE e.department != 'Legal'  -- Exclude Legal for compliance reasons
""")
print("✓ Created data_analyst_view: Anonymous IDs + full compensation data (enhanced anonymization)")

✓ Created data_analyst_view: Anonymous IDs + full compensation data (enhanced anonymization)


### **Configure Group Permissions**

Now we'll grant permissions to the `Dev` group you have created (which is the group that you've added the service principal `HR_data_analyst` to -- in order to inherit the groups permission). 

This implements the principle of least privilege - the group only gets access to anonymized, aggregated data, never raw employee records.

In [0]:
# Verify the Devs group exists
groups_df = spark.sql("SHOW GROUPS")
display(groups_df)
print("✅ Confirmed: Devs group is available")

name,directGroup
admins,null
Devs,null
users,null
test-shubham,null
deployment_devs,null


✅ Confirmed: Devs group is available


In [0]:
# Confirm view in the schema
print("📋 Verifying our governance view was created successfully...")
view = spark.sql(f"SHOW VIEWS IN {catalog_name}.{schema_name}")
display(view)

print("\n✅ You should see:")
print("   - data_analyst_view (for agents and analysts)")

📋 Verifying our governance view was created successfully...


namespace,viewName,isTemporary,isMaterialized,isMetric
hr_data,data_analyst_view,false,false,false



✅ You should see:
   - data_analyst_view (for agents and analysts)


In [0]:
group_name = "Devs"

# Grant broad permissions at catalog level
spark.sql(f"GRANT ALL PRIVILEGES ON CATALOG {catalog_name} TO {group_name}")

print(f"✅ Granted ALL PRIVILEGES on catalog {catalog_name} to {group_name}")
print("This should inherit to all schemas, tables, view, and models in the catalog")

✅ Granted ALL PRIVILEGES on catalog clientcare to Devs
This should inherit to all schemas, tables, view, and models in the catalog


### **Column Masking**
**Why implement this when we already have views?**

Views can be bypassed if users gain direct table access. Column masking at the table level cannot be bypassed - it's enforced by Unity Catalog on every query, regardless of how the data is accessed.

**How Table-Level Security Works:**
- **Column Masking**: Functions run automatically transforming sensitive data based on who's viewing it
- **Row-Level Security**: Functions can also control which rows users see (we already handled Legal department filtering in our views, but this could be done at the table level too)
- Unity Catalog enforces these at the storage layer - users **cannot bypass** them
- Different users see different versions of the same data

In [0]:
# Implement column masking for SSN field - Defense in depth security
print("🔐 Implementing column masking for SSN field...")

# Create column masking function for SSN
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog_name}.{schema_name}.mask_ssn(ssn_value STRING)
RETURNS STRING
RETURN CASE 
    WHEN is_account_group_member('hr_data_analysts') THEN 'ANALYTICS_MASKED'
    ELSE CONCAT('***-**-', RIGHT(ssn_value, 4))
END
""")

# Apply the masking function to the SSN column
spark.sql(f"""
ALTER TABLE {catalog_name}.{schema_name}.employee_records 
ALTER COLUMN ssn 
SET MASK {catalog_name}.{schema_name}.mask_ssn
""")

print("✅ Column masking applied to SSN field")
print("🔐 Defense in Depth: SSN now masked at table level")
print("   - Admins see full SSN")
print("   - hr_data_analysts group sees 'ANALYTICS_MASKED'") 
print("   - Other users see '***-**-1234' format")

🔐 Implementing column masking for SSN field...
✅ Column masking applied to SSN field
🔐 Defense in Depth: SSN now masked at table level
   - Admins see full SSN
   - hr_data_analysts group sees 'ANALYTICS_MASKED'
   - Other users see '***-**-1234' format


In [0]:
# Validate column masking implementation
print("🧪 Testing SSN column masking...")

print("\n👤 Current user view:")
display(spark.sql(f"""
SELECT employee_id, first_name, last_name, ssn, department 
FROM {catalog_name}.{schema_name}.employee_records 
LIMIT 5
"""))


🧪 Testing SSN column masking...

👤 Current user view:


employee_id,first_name,last_name,ssn,department
E001,John,Smith,***-**-6789,Engineering
E002,Sarah,Johnson,***-**-7890,Sales
E003,Michael,Brown,***-**-8901,HR
E004,Emily,Davis,***-**-9012,Engineering
E005,Robert,Wilson,***-**-0123,Marketing


### **Build Secure Tools for the Agent**
Now we'll create Unity Catalog functions that our AI agent will use to query HR data. These functions act as the interface between the agent and our governed data.

**Why UC Functions for Agent Tools?**
- **Governed Access**: Functions inherit caller permissions 
- **Query Templates**: They validate inputs and enforce safe query patterns
- **Audit Trail**: All function calls are logged for compliance
- **Performance**: Functions can be optimized with proper indexing and caching

**Our Agent Tool Strategy:**
We'll create two general-purpose functions that:
1. Work exclusively with the anonymized `data_analyst_view`
2. Return aggregated results that can answer various HR questions
3. Prevent the agent from writing arbitrary SQL
4. Maintain employee privacy through anonymous IDs

In [0]:
# TOOL 1: Performance & Retention Analytics
print("🔧 Creating performance analytics function...")

spark.sql(f"""
    CREATE OR REPLACE FUNCTION
    {catalog_name}.{schema_name}.analyze_performance()
    RETURNS TABLE (
        department STRING,
        avg_rating DOUBLE,
        min_rating DOUBLE,
        max_rating DOUBLE,
        employee_count INT,
        avg_tenure_years DOUBLE
    )
    COMMENT 'HR Analytics: Basic performance metrics by department'
    RETURN (
        SELECT 
            department,
            AVG(rating) as avg_rating,
            MIN(rating) as min_rating,
            MAX(rating) as max_rating,
            COUNT(DISTINCT anonymous_id) as employee_count,
            AVG(YEAR(CURRENT_DATE()) - hire_year) as avg_tenure_years
        FROM {catalog_name}.{schema_name}.data_analyst_view
        WHERE rating IS NOT NULL
        GROUP BY department
    );
""")

print("✓ Created analyze_performance function")
print("  - Returns: avg/min/max ratings, employee count, avg tenure by department")
print("  - Works with: anonymized data only")

🔧 Creating performance analytics function...
✓ Created analyze_performance function
  - Returns: avg/min/max ratings, employee count, avg tenure by department
  - Works with: anonymized data only


In [0]:
# TOOL 2: Department & Compensation Analytics
print("🔧 Creating operations analytics function...")

spark.sql(f"""
    CREATE OR REPLACE FUNCTION
    {catalog_name}.{schema_name}.analyze_operations()
    RETURNS TABLE (
        department STRING,
        employee_count INT,
        avg_salary DOUBLE,
        avg_bonus DOUBLE,
        avg_total_comp DOUBLE,
        avg_stock_options INT
    )
    COMMENT 'HR Analytics: Department compensation and operational metrics'
    RETURN (
        SELECT 
            department,
            COUNT(DISTINCT anonymous_id) as employee_count,
            AVG(base_salary) as avg_salary,
            AVG(bonus) as avg_bonus,
            AVG(base_salary + bonus) as avg_total_comp,
            AVG(stock_options) as avg_stock_options
        FROM {catalog_name}.{schema_name}.data_analyst_view
        WHERE base_salary IS NOT NULL
        GROUP BY department
    );
""")

print("✓ Created analyze_operations function")
print("  - Returns: compensation metrics and headcount by department")
print("  - Maintains privacy: no individual employee data exposed")


🔧 Creating operations analytics function...
✓ Created analyze_operations function
  - Returns: compensation metrics and headcount by department
  - Maintains privacy: no individual employee data exposed


In [0]:
# Test 1: Performance Analytics
print("\n📊 Test 1: Performance Analytics by Department")
display(spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.analyze_performance()"))

# Test 2: Operations/Compensation Analytics
print("\n📊 Test 2: Compensation Analytics by Department")
display(spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.analyze_operations()"))


📊 Test 1: Performance Analytics by Department


department,avg_rating,min_rating,max_rating,employee_count,avg_tenure_years
HR,4.25,4.2,4.3,2,5.0
Sales,3.8499999999999996,3.8,3.9,2,5.0
Finance,4.4,4.4,4.4,1,5.0
Marketing,3.85,3.7,4.0,2,4.0
Engineering,4.6000000000000005,4.5,4.7,3,3.3333333333333335



📊 Test 2: Compensation Analytics by Department


department,employee_count,avg_salary,avg_bonus,avg_total_comp,avg_stock_options
HR,2,93500.0,9750.0,103250.0,2900
Sales,2,86500.0,11750.0,98250.0,2100
Finance,1,115000.0,16000.0,131000.0,4500
Marketing,2,88500.0,10750.0,99250.0,2450
Engineering,3,111666.66666666667,14000.0,125666.66666666667,4166


## ✅ Governance Foundation Complete

### What We've Accomplished:
1. **Data Classification with Tagging**: Applied sensitivity labels (Public, Internal, Confidential, Restricted) to all HR tables

2. **Built Classification-Aware View**: 
   - `data_analyst_view` - Anonymous IDs, year-only dates, full compensation (designed for AI agents)
   
3. **Configured Group-Based Access**:
   - Set up `Dev` group with permissions on our view and functions
   - Implemented enterprise-standard permission management
   - Ready for both users and service principals

4. **Implemented Multi-Layer Security**: 
   - Table-level SSN masking (cannot be bypassed)
   - Row-level filtering (Legal department excluded in our view)
   - Column-level anonymization (employee IDs → EMP_000001 format)

5. **Built Secure Agent Tools**:
   - `analyze_performance()` - Returns performance ratings and tenure by department
   - `analyze_operations()` - Returns compensation metrics and headcount by department
   - Both functions granted to `Dev` group


### The governance foundation is ready. Let's build the AI agent in Lab 2!